In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn import linear_model
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics.pairwise import sigmoid_kernel, rbf_kernel, polynomial_kernel
from sklearn import neighbors
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


## 1. Load Data

In [2]:
# Load the features
X = np.loadtxt(open("AD_MRI_DATA/2-CN_MCI/2.3-MRI_Volume.csv","rb"),delimiter=",",skiprows=0)
X = np.array(X, dtype=float)
print("X:" + str(X.shape))

# Load the labels
with open('AD_MRI_DATA/2-CN_MCI/2-Phenotype.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    c1 = [row[1]for row in reader]
    
del c1[0]
y = np.array(c1, dtype=int)
print("y:" + str(y.shape))


X:(188, 109)
y:(188,)


## 2. Lasso with Classifiers

### 2.1 Lasso

In [3]:
model = linear_model.LassoCV(alphas=[1,0.1,0.01,0.005,0.001,0.0001])
#alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X, y)

print("alpha: " + str(model.alpha_))
#print(lasso.coef_)
3
selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected = selector.transform(X)
print("X_selected: " + str(X_selected.shape))

/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.170719673367839, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.117547586102379, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.703986242902368, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda

alpha: 1.0
X_selected: (188, 75)


/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.030973108885284262, tolerance: 0.0037615894039735092
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.445989130279918, tolerance: 0.0037615894039735092
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.95975431643632, tolerance: 0.0037615894039735092
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/ana

### 2.2 Classifiers

In [4]:
for i in range(5):
    # SVM
    if i == 0:
        clf = SVC(C=1, random_state=42, probability=True)
        print("SVM")
        
        
    # KNN
    elif i == 1:
        clf = neighbors.KNeighborsClassifier(n_neighbors=2)
        print("KNN")
        
    # Gaussian Naive Bayes
    elif i == 2:
        clf = GaussianNB()
        print("Gaussian Naive Bayes")
        
    # Decision Tree
    elif i == 3:
        clf = tree.DecisionTreeClassifier()
        print("Decision Tree")
        
    # Random Forest
    else:
        clf = RandomForestClassifier()
        print("Random Forest")
    
    #模型评价
    y_pred = cross_val_predict(clf, X_selected, y, cv=5)
    auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
    print(confusion_matrix(y, y_pred,labels=[1,0]))
    print(classification_report(y, y_pred))
    auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
    print("auc: "+str(auc_avg)+"\n")


SVM
[[100   0]
 [ 88   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        88
           1       0.53      1.00      0.69       100

    accuracy                           0.53       188
   macro avg       0.27      0.50      0.35       188
weighted avg       0.28      0.53      0.37       188

auc: 0.5357516339869282

KNN
[[18 82]
 [14 74]]
              precision    recall  f1-score   support

           0       0.47      0.84      0.61        88
           1       0.56      0.18      0.27       100

    accuracy                           0.49       188
   macro avg       0.52      0.51      0.44       188
weighted avg       0.52      0.49      0.43       188

auc: 0.476813725490196

Gaussian Naive Bayes
[[53 47]
 [34 54]]
              precision    recall  f1-score   support

           0       0.53      0.61      0.57        88
           1       0.61      0.53      0.57       100

    accuracy                           0.57

/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[62 38]
 [37 51]]
              precision    recall  f1-score   support

           0       0.57      0.58      0.58        88
           1       0.63      0.62      0.62       100

    accuracy                           0.60       188
   macro avg       0.60      0.60      0.60       188
weighted avg       0.60      0.60      0.60       188

auc: 0.6650490196078431



### 2.3 Multi-core SVM

In [5]:
sig_mat = sigmoid_kernel(X_selected)
rbf_mat = rbf_kernel(X_selected)
poly_mat = polynomial_kernel(X_selected)

best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        train_kernel = w1 * sig_mat + w2 * rbf_mat + w3 * poly_mat
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, train_kernel, y, cv=5, scoring="accuracy")
        acc_avg = (acc[0] + acc[1] + acc[2])/3
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.0 w2:0.0 w3:1.0
score:0.5964912280701754


In [8]:
train_kernel = best_w1 * sig_mat + best_w2 * rbf_mat + best_w3 * poly_mat
y_pred = cross_val_predict(clf, train_kernel, y, cv=5)
auc = cross_val_score(clf, train_kernel, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
print("auc: "+str(auc_avg)+"\n")

[[53 47]
 [40 48]]
              precision    recall  f1-score   support

           0       0.51      0.55      0.52        88
           1       0.57      0.53      0.55       100

    accuracy                           0.54       188
   macro avg       0.54      0.54      0.54       188
weighted avg       0.54      0.54      0.54       188

auc: 0.5140849673202614



## 3. ElasticNet with Classifiers

### 3.1 ElasticNet

In [6]:
model = ElasticNetCV(random_state=0,alphas=[1,0.1,0.01,0.005,0.001,0.0001])
#,alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X, y)

print("alpha: " + str(model.alpha_))
#print(els.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected = selector.transform(X)
print("X_selected: " + str(X_selected.shape))

/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.36916133130322, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.918583959718067, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.6688259054505634, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda

alpha: 1.0
X_selected: (188, 33)


/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.229374438482259, tolerance: 0.0037549668874172193
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.4628247966408017, tolerance: 0.0037615894039735092
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.224616734040139, tolerance: 0.0037615894039735092
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anac

### 3.2 Classifiers

In [7]:
for i in range(5):
    # SVM
    if i == 0:
        clf = SVC(C=1, random_state=42, probability=True)
        print("SVM")
        
        
    # KNN
    elif i == 1:
        clf = neighbors.KNeighborsClassifier(n_neighbors=2)
        print("KNN")
        
    # Gaussian Naive Bayes
    elif i == 2:
        clf = GaussianNB()
        print("Gaussian Naive Bayes")
        
    # Decision Tree
    elif i == 3:
        clf = tree.DecisionTreeClassifier()
        print("Decision Tree")
        
    # Random Forest
    else:
        clf = RandomForestClassifier()
        print("Random Forest")
    
    # Model Evaluation
    y_pred = cross_val_predict(clf, X_selected, y, cv=5)
    auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
    print(confusion_matrix(y, y_pred,labels=[1,0]))
    print(classification_report(y, y_pred))
    auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
    print("auc: "+str(auc_avg)+"\n")

SVM
[[100   0]
 [ 88   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        88
           1       0.53      1.00      0.69       100

    accuracy                           0.53       188
   macro avg       0.27      0.50      0.35       188
weighted avg       0.28      0.53      0.37       188

auc: 0.7100980392156864

KNN
[[24 76]
 [16 72]]
              precision    recall  f1-score   support

           0       0.49      0.82      0.61        88
           1       0.60      0.24      0.34       100

    accuracy                           0.51       188
   macro avg       0.54      0.53      0.48       188
weighted avg       0.55      0.51      0.47       188

auc: 0.5829084967320262

Gaussian Naive Bayes
[[63 37]
 [32 56]]
              precision    recall  f1-score   support

           0       0.60      0.64      0.62        88
           1       0.66      0.63      0.65       100

    accuracy                           0.6

/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[64 36]
 [39 49]]
              precision    recall  f1-score   support

           0       0.58      0.56      0.57        88
           1       0.62      0.64      0.63       100

    accuracy                           0.60       188
   macro avg       0.60      0.60      0.60       188
weighted avg       0.60      0.60      0.60       188

auc: 0.6197385620915032



### 3.3 Multi-core SVM

In [9]:
sig_mat = sigmoid_kernel(X_selected)
rbf_mat = rbf_kernel(X_selected)
poly_mat = polynomial_kernel(X_selected)

best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        train_kernel = w1 * sig_mat + w2 * rbf_mat + w3 * poly_mat
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, train_kernel, y, cv=5, scoring="accuracy")
        acc_avg = (acc[0] + acc[1] + acc[2])/3
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.0 w2:0.0 w3:1.0
score:0.5701754385964911


In [10]:
train_kernel = best_w1 * sig_mat + best_w2 * rbf_mat + best_w3 * poly_mat
y_pred = cross_val_predict(clf, train_kernel, y, cv=5)
auc = cross_val_score(clf, train_kernel, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
print("auc: "+str(auc_avg)+"\n")

[[60 40]
 [43 45]]
              precision    recall  f1-score   support

           0       0.53      0.51      0.52        88
           1       0.58      0.60      0.59       100

    accuracy                           0.56       188
   macro avg       0.56      0.56      0.56       188
weighted avg       0.56      0.56      0.56       188

auc: 0.5899019607843137

